### Data_loader.ipynb
### Author: moutaz elias

Data_loader.ipynb takes care of loading the test and training data, explaratory analysis, cleaning the data (one hot encoding, feature extraction, and fill Nan), and spliting the training data into test and train k fold sets.

In [40]:
#importing relevant libraries
#importing relevant libraries
import numpy as np
import scipy as sp
import pandas as pd
import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split as split
from sklearn.impute import SimpleImputer
import os as os
import pickle as pkl
import missingno as msno
import matplotlib.pyplot as plt

In [2]:
#Turning off warning and other jupyter specific options
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
#data directories and checking if they exist
directory_train="../data/exercise_20_train.csv"
directory_test="../data/exercise_20_test.csv"

if not os.path.isfile(directory_train):
    print("Training file does not exist")
if not os.path.isfile(directory_test):
    print("Test file does not exist")

In [4]:
#loading data
raw_train_df=pd.read_csv(directory_train)
raw_test_df=pd.read_csv(directory_test)

#from this point I will deal with train data first
#inspect train data headings
print(raw_train_df.dtypes.value_counts())
raw_train_df.head(5)


float64    94
object      6
int64       1
dtype: int64


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,0.963686,6.627185,-45.224008,9.477531,-3.216532,13.216874,9.754747,5.245851,-1.102918,-2.867482,...,0.988829,0.313772,asia,1.380664,-16.388994,5.326730,4.187294,0.045549,-3.646841,0
1,-1.770062,-23.610459,-0.964003,-31.981497,-10.294599,-10.240251,-1.518888,-1.675208,0.498134,-0.614390,...,-2.162863,1.809807,asia,2.500590,4.338834,-1.583225,-1.172417,0.011216,0.097180,0
2,9.962401,-8.349849,23.248891,-24.196879,8.937480,10.965000,-7.490596,-3.025094,0.595807,0.382732,...,1.779660,9.528113,asia,1.396475,7.839188,10.402396,1.288991,0.008209,-4.132316,0
3,-5.780709,-25.261584,1.383115,-11.786929,7.993078,-11.245752,-2.607351,-3.513896,-0.614235,-1.453979,...,-0.203206,4.892248,asia,0.744317,7.380982,7.599323,-8.022884,-0.067624,-1.796198,0
4,1.211541,1.119963,7.512938,21.987312,-5.155392,10.339416,3.045180,-0.619230,-0.928068,0.405024,...,0.248724,18.694990,asia,1.703196,-11.552129,0.381768,-3.550471,-0.055180,-3.344490,0


99 rows unamed rows and 1 y class prediciton row.<br>
  No information about rows provided so hard to provide intuition or domain expertise.<br>
  Contains both categorical and numerical variables.<br>


In [5]:
#divide into categorical and numerical
num_train_df=raw_train_df.loc[:,raw_train_df.dtypes == np.float64]
cat_train_df=raw_train_df.loc[:,raw_train_df.dtypes == np.object]
y_train_df=raw_train_df["y"].astype(int)

In [6]:
#take a look at categorical data
for cat in cat_train_df:
    print(cat_train_df[cat].value_counts(),'\n')
    
cat_train_df.describe()

volkswagon    12622
Toyota        10968
bmw            7262
Honda          5174
tesla          2247
chrystler      1191
nissan          326
ford            160
mercades         31
chevrolet        11
Name: x34, dtype: int64 

wed          14820
thurday      13324
wednesday     5938
thur          4428
tuesday        884
friday         517
monday          53
fri             26
Name: x35, dtype: int64 

$156.29     4
$680.85     4
$-370.55    4
$-511.36    4
$732.95     3
           ..
$-644.69    1
$624.06     1
$-269.37    1
$-223.71    1
$-751.03    1
Name: x41, Length: 37817, dtype: int64 

0.01%     9610
-0.01%    9547
0.0%      7876
-0.0%     7674
0.02%     2373
-0.02%    2363
-0.03%     279
0.03%      243
-0.04%      17
0.04%       11
Name: x45, dtype: int64 

July       11114
Jun         9317
Aug         8170
May         4744
sept.       3504
Apr         1629
Oct          885
Mar          407
Nov          145
Feb           48
Dev           16
January       12
Name: x68, dtype: int

,x34,x35,x41,x45,x68,x93
count,39992,39990,39996,39993,39991,39993
unique,10,8,37817,10,12,3
top,volkswagon,wed,$156.29,0.01%,July,asia
freq,12622,14820,4,9610,11114,35384


In [7]:
#replacing values that are interchangable
#chose to replace with wed fri thur as the other way around will create wedwednesday... wed is part of wednesday 
cat_train_df['x35'] = cat_train_df['x35'].replace('wednesday','wed', regex = True)
cat_train_df['x35'] = cat_train_df['x35'].replace('friday','fri', regex = True)
cat_train_df['x35'] = cat_train_df['x35'].replace('thurday','thur', regex = True)

# has no real effect just for cosmetic reasons but fixing other cat values
cat_train_df['x68'] = cat_train_df['x68'].replace('sept.','Sept', regex = True)
cat_train_df['x68'] = cat_train_df['x68'].replace('January','Jan', regex = True)

In [8]:
## remaining of these are not actually categorical data.
## converting to true numerical data and adding to raw_num_train_df
num_train_df['x41'] = cat_train_df['x41'].replace('[/$]','', regex = True).astype(float)
num_train_df['x45'] = cat_train_df['x45'].replace('[%]','', regex = True).astype(float)

In [9]:
#dropping out of categorical
cat_train_df.drop(['x41'],axis=1,inplace=True)
cat_train_df.drop(['x45'],axis=1,inplace=True)

In [10]:
#taking a look at cat variables pre label and one hot encoding
cat_train_df.head(5)

,x34,x35,x68,x93
0,chrystler,thur,Sept,asia
1,volkswagon,thur,July,asia
2,bmw,thur,July,asia
3,nissan,thur,July,asia
4,volkswagon,wed,Jun,asia


In [11]:
#visualize missing num and cat data first
#msno.matrix(num_train_df)
#print("number of rows with missing values")
#print(sum(num_train_df.apply(lambda x: sum(x.isnull().values), axis = 1)>0)+
#     sum(cat_train_df.apply(lambda x: sum(x.isnull().values), axis = 1)>0))

#msno.matrix(cat_train_df)


 Data is pretty clean low percentage of Nan in rows.<br>
 806 out of 40000 or 2%<br>
 As instructions specifically ask to avoid data leakage imputation of numerical values <br>
 Performing splitting into test train before imputation preventing data leakage.<br>
 Categorical values will be simply filles with most common instance no need for splitting<br>

In [12]:
#substituting Nan in categorical data with most frequent data 
#Checked that NaN was not the most frequent no need to split as only 34 missing NaN and no
#data leakage
for cat in cat_train_df:
    cat_train_df[cat].fillna(cat_train_df[cat].value_counts().index[0],inplace=True)

In [13]:
#imputing and scaling numerical data
#imputer will only be used to fir training part of the training model
#It will be applied to Train_train, Train_test,Test
#same process will be done to scaling
semi_raw_df=pd.concat([num_train_df, cat_train_df], axis = 1)
x_train1,x_val1,y_train,y_val=split(semi_raw_df,y_train_df,test_size=0.2,random_state=1)

In [14]:
#Spliting into trained into cat and numerical before data prep and clean
num_x_train_df=x_train1.loc[:,x_train1.dtypes == np.float64]
cat_X_train_df=x_train1.loc[:,x_train1.dtypes == np.object]


num_x_val_df=x_val1.loc[:,x_val1.dtypes == np.float64]
cat_X_val_df=x_val1.loc[:,x_val1.dtypes == np.object] 

In [15]:
#Converting into nominal values
Cat_dict={}
one_labels_df=pd.DataFrame()
Encoder = pre.OneHotEncoder()

for cat in cat_X_train_df:
    Lab_Encoder = pre.LabelEncoder()
    one_labels_df[cat]= Lab_Encoder.fit_transform(cat_X_train_df[cat])
    Cat_dict[cat]= Lab_Encoder
    
#One hot encoding
one_train_df  = pd.DataFrame(Encoder.fit_transform(one_labels_df).toarray())

In [16]:
#numerical values imputation using SimpleImpute
imp = SimpleImputer()
imp_num_x_train_df=pd.DataFrame(imp.fit_transform(num_x_train_df))

#scaling num value
scaling= pre.StandardScaler()
scaled_num_x_train_df = pd.DataFrame(scaling.fit_transform(imp_num_x_train_df))

In [17]:
#Creating x_train and pkl dumping x_train y train
x_train=pd.concat([scaled_num_x_train_df,one_train_df],axis=1)

pickle_out = open("x_train.pickle","wb")
pkl.dump(x_train, pickle_out)
pickle_out.close()

pickle_out = open("y_train.pickle","wb")
pkl.dump(y_train, pickle_out)
pickle_out.close()

In [18]:
#one hot encoding the validation
one_val_labels_df=pd.DataFrame()
for cat in Cat_dict:
    one_val_labels_df[cat]=Cat_dict[cat].transform(cat_X_val_df[cat])

one_val_df=pd.DataFrame(Encoder.transform(one_val_labels_df).toarray())

In [19]:
#Imputing the Validation data set
imp_num_x_val_df=pd.DataFrame(imp.transform(num_x_val_df))

scaled_num_x_val_df = pd.DataFrame(scaling.transform(imp_num_x_val_df))

In [20]:
#Creating x_val and pkl dumping x_val y_val
x_val=pd.concat([scaled_num_x_val_df,one_val_df],axis=1)

pickle_out = open("x_val.pickle","wb")
pkl.dump(x_val, pickle_out)
pickle_out.close()

pickle_out = open("y_val.pickle","wb")
pkl.dump(y_val, pickle_out)
pickle_out.close()

## This section will repeat the process for the test data.

In [21]:
# An assumption is that test and train data are identical
print(raw_test_df.describe())
#divide into categorical and numerical
num_test_df=raw_test_df.loc[:,raw_test_df.dtypes == np.float64]
cat_test_df=raw_test_df.loc[:,raw_test_df.dtypes == np.object]


                x0           x1           x2            x3           x4  \
count  9998.000000  9998.000000  9998.000000  10000.000000  9999.000000   
mean      2.123389    -4.170347     0.429555     -1.670248     0.492788   
std       9.689917    18.696344    21.030355     29.335529    20.076053   
min     -35.502970   -79.642519   -78.549288   -122.923518   -75.801406   
25%      -4.544092   -16.505208   -13.378986    -21.568207   -13.099560   
50%       2.180901    -4.349082     0.474530     -1.660928     0.481522   
75%       8.611017     8.366082    14.719134     18.196968    13.944449   
max      43.060198    65.821750    75.474024    114.318448    76.851984   

                x5           x6           x7           x8           x9  ...  \
count  9997.000000  9998.000000  9997.000000  9999.000000  9998.000000  ...   
mean     -0.860844     0.051060    -0.016683    -0.363496     0.005506  ...   
std      18.379462     6.784101     5.646885     1.631196     1.355688  ...   
min     

In [22]:
#replacing values that are interchangable
#chose to replace with wed fri thur as the other way around will create wedwednesday... wed is part of wednesday 
cat_test_df['x35'] = cat_test_df['x35'].replace('wednesday','wed', regex = True)
cat_test_df['x35'] = cat_test_df['x35'].replace('friday','fri', regex = True)
cat_test_df['x35'] = cat_test_df['x35'].replace('thurday','thur', regex = True)

# has no real effect just for cosmetic reasons but fixing other cat values
cat_test_df['x68'] = cat_test_df['x68'].replace('sept.','Sept', regex = True)
cat_test_df['x68'] = cat_test_df['x68'].replace('January','Jan', regex = True)

## remaining of these are not actually categorical data.
## converting to true numerical data and adding to raw_num_train_df
num_test_df['x41'] = cat_test_df['x41'].replace('[/$]','', regex = True).astype(float)
num_test_df['x45'] = cat_test_df['x45'].replace('[%]','', regex = True).astype(float)

#dropping out of categorical
cat_test_df.drop(['x41'],axis=1,inplace=True)
cat_test_df.drop(['x45'],axis=1,inplace=True)

In [23]:
#substituting Nan in categorical data with most frequent data 
for cat in cat_test_df:
    cat_test_df[cat].fillna(cat_test_df[cat].value_counts().index[0],inplace=True)

In [24]:
print(cat_test_df['x35'].value_counts(dropna=True))

wed        5199
thur       4408
tuesday     228
fri         144
monday       21
Name: x35, dtype: int64


In [25]:
#Converting into nominal values
one_labels_test_df=pd.DataFrame()

for cat in cat_test_df:
    one_labels_test_df[cat]=Cat_dict[cat].transform(cat_test_df[cat])

one_test_df=pd.DataFrame(Encoder.transform(one_labels_test_df).toarray())

In [26]:
#Imputing the testing data set
imp_num_x_test_df=pd.DataFrame(imp.transform(num_test_df))

#scaling
scaled_num_x_test_df = pd.DataFrame(scaling.transform(imp_num_x_test_df))

In [27]:
#Creating x_test and pkl dumping x_test y_test
x_test=pd.concat([scaled_num_x_test_df,one_test_df],axis=1)

pickle_out = open("x_test.pickle","wb")
pkl.dump(x_test, pickle_out)
pickle_out.close()


# end of file

In [28]:
error=pd.DataFrame((num_x_val_df.mean()-num_train_df.mean())/num_x_val_df.mean()*100)

In [29]:
error.median()

0    17.961966
dtype: float64

In [134]:
#checking correlaitons 
#and multicoriniarity 

correlated_data=x_train.corr().abs()
corrMatrix=correlated_data[correlated_data>0.8].unstack().dropna(how="all").drop_duplicates()

In [135]:
print(corrMatrix)

0   0     1.000000
12  14    0.927503
27  28    0.810152
dtype: float64
